# Feladat: Genetikus Algoritmus - Rendszámok Elrejtése
Egy ország új rendszámformátumot szeretne bevezetni, amely 7 karakterből áll: az első három pozíció betű (A-Z), az utolsó négy pozíció szám (0-9). Például egy ilyen rendszám lehet: ABC1234.

A cél az, hogy olyan rendszámokat generáljunk, amelyek:
  1.	Minél könnyebben megjegyezhetők (például, ha tartalmaz ismétlődő karaktereket).
  2.	Specifikus mintázatot követnek, például az utolsó számjegy 4 vagy 7.
  A genetikus algoritmust használjuk arra, hogy a rendszámokat generáljuk és finomítsuk a megadott cél érdekében.

Célok:
  1.	Fitness függvény: Minden egyes rendszámot a következő kritériumok alapján értékelj:
    *	Minél több ismétlődő karakter szerepel a rendszámban, annál jobb.
    *	Az utolsó számjegy 4 vagy 7 legyen.
    *	Ha az első három betű AAA, BBB vagy CCC, akkor + bónusz pont.
  2.	Keresztezés és mutáció:
    *	Kombinálj két rendszámot, hogy újakat hozz létre.
    *	Véletlenszerűen változtass meg egy karaktert mutációval.
  3.	Populáció: Kezdj véletlenszerű rendszámokkal, és futtasd a genetikus algoritmust, amíg egy megfelelő rendszámot nem találunk.



In [1]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.4 MB/s eta 0:00:00


In [2]:
import random
from deap import base, creator, tools, algorithms

# Fitness függvény definiálása
def evaluate_license(license_plate):
  score = 0

  # Kritérium: ismétlődő karakterek
  score += len(license_plate) - len(set(license_plate))

  # Kritérium: utolsó számjegy 4 vagy 7
  if license_plate[-1] in "47":
    score += 3

  # Kritérium: első három betű "AAA", "BBB", "CCC"
  if ''.join(license_plate[:3]) in ["AAA", "BBB", "CCC"]:
    score += 5

  return (score,)

# Genetikus algoritmus inicializálása
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Véletlenszerű rendszám generálása
def random_license():
  letters = ''.join(random.choices("ABCDEFGHIJKLMNOPQRSTUVWXYZ", k=3))
  numbers = ''.join(random.choices("0123456789", k=4))
  return list(letters + numbers)

# Egyén és populáció létrehozása
toolbox.register("individual", tools.initIterate, creator.Individual, random_license)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetikus operátorok
toolbox.register("evaluate", evaluate_license)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=35, indpb=0.1)  # Mutáció: véletlen módosítás
toolbox.register("select", tools.selTournament, tournsize=3)

# Dekódolás a karakterek és számok között
def decode_license(license_plate):
  return ''.join(license_plate)

# Véletlen mutáció adaptálása
def custom_mutate(individual):
  index = random.randint(0, len(individual) - 1)
  if index < 3:  # Betű
    individual[index] = random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
  else:  # Szám
    individual[index] = random.choice("0123456789")
  return individual,

toolbox.register("mutate", custom_mutate)

# Evolúció futtatása
def run_evolution(pop_size=100, generations=50):
  population = toolbox.population(n=pop_size)

  # Statisztika Elemzése:
  # A stats objektum most a fitness érték első eleme alapján működik:
  # Stat útján követhetjük a fejlődést
  stats = tools.Statistics(lambda ind: ind.fitness.values[0])  # Fitness tuple első eleme

  # Átlag Számítás:
  # Az avg a tuple első értékeinek átlagát számolja
  stats.register("avg", lambda x: sum(x) / len(x))  # Átlag
  stats.register("max", max)  # Maximum érték

  hof = tools.HallOfFame(1)  # Legjobb egyed tárolása

  population, logbook = algorithms.eaSimple(
    population,
    toolbox,
    cxpb=0.7,
    mutpb=0.2,
    ngen=generations,
    stats=stats,
    halloffame=hof,
    verbose=True
  )

  return population, logbook, hof

# Kód futtatása
if __name__ == "__main__":
  population, logbook, hof = run_evolution()

  print("\nLegjobb rendszám:", decode_license(hof[0]))
  print("Fitness érték:", hof[0].fitness.values[0])


gen	nevals	avg 	max
0  	100   	1.23	5  
1  	80    	2.02	4  
2  	80    	3.25	5  
3  	76    	3.8 	6  
4  	72    	4.03	6  
5  	84    	4.2 	6  
6  	79    	4.64	6  
7  	86    	4.91	6  
8  	80    	5.11	6  
9  	81    	5.24	6  
10 	73    	5.48	6  
11 	75    	5.76	6  
12 	80    	5.7 	6  
13 	72    	5.84	7  
14 	77    	5.85	7  
15 	81    	5.74	7  
16 	80    	5.79	7  
17 	84    	5.96	7  
18 	74    	6.21	7  
19 	75    	6.59	8  
20 	82    	6.73	8  
21 	82    	6.78	8  
22 	69    	6.89	8  
23 	71    	7.1 	8  
24 	77    	7.34	8  
25 	72    	7.46	8  
26 	84    	7.55	8  
27 	88    	7.68	8  
28 	84    	7.87	8  
29 	84    	7.61	8  
30 	74    	7.75	8  
31 	75    	7.76	8  
32 	77    	7.78	8  
33 	76    	7.81	8  
34 	78    	7.78	8  
35 	72    	7.71	8  
36 	78    	7.71	8  
37 	72    	7.79	8  
38 	77    	7.75	8  
39 	73    	7.73	8  
40 	72    	7.67	8  
41 	77    	7.72	8  
42 	80    	7.76	8  
43 	85    	7.8 	8  
44 	78    	7.72	8  
45 	73    	7.83	8  
46 	80    	7.75	8  
47 	81    	7.65	8  
48 	79    	7.71	8  


Az evolúciós algoritmus futása során a gen, nevals, avg és max oszlopok a populáció fejlődését követik nyomon az egyes generációkban.

Az oszlopok jelentése:
1. gen (Generation):
  * Az aktuális generáció száma. Az evolúció az 1. generációból indul, és a megadott ngen (pl. 50) generációszámig fut.

2. nevals (Number of Evaluations):
  * Az adott generációban kiértékelt egyedek (individuális megoldások) száma. Ez az algoritmus által meghívott evaluate függvények számlálója.

3. avg (Average Fitness):
  * Az adott generáció összes egyedének átlagos fitness értéke. Ez mutatja, hogy a populáció általános teljesítménye hogyan változik az idő előrehaladtával.

4. max (Maximum Fitness):
  * Az adott generáció legjobb egyedének fitness értéke. Ez az oszlop követi, hogy az algoritmus legjobb megoldása hogyan javul a generációk során.